## fastText chABSA dataset slot 3 evaluation

Tested single data split only, not cross validation applied.

### Polarity classification

    P@1	0.751
    R@1	0.751
    Number of examples: 265

### Category classification

    P@1	0.464
    R@1	0.464
    Number of examples: 265

In [1]:
! git clone https://github.com/daisukelab/dl-cliche.git
! cd dl-cliche && pip install .
! rm -fr dl-cliche

Cloning into 'dl-cliche'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 141 (delta 83), reused 93 (delta 40), pack-reused 0
Receiving objects: 100% (141/141), 31.97 KiB | 0 bytes/s, done.
Resolving deltas: 100% (83/83), done.
Checking connectivity... done.
Processing /home/xxxx/lab/NLP/bert/dl-cliche
  Running setup.py bdist_wheel for dl-cliche ... done
  Stored in directory: /home/xxxx/.cache/pip/wheels/75/2e/09/2ade8d86933e056eb5238298b5b39f70d99ba03f4b9ae35df6
Successfully built dl-cliche
  Found existing installation: dl-cliche 0.0.1
    Uninstalling dl-cliche-0.0.1:
      Successfully uninstalled dl-cliche-0.0.1


In [1]:
from dlcliche.utils import *
from dlcliche.nlp_mecab import *

In [4]:
# Download dataset & stop_words_ja.txt
! wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
! unzip -q chABSA-dataset.zip && rm chABSA-dataset.zip && rm -r __MACOSX
! ls chABSA-dataset
! cd chABSA-dataset && wget https://raw.githubusercontent.com/chakki-works/chABSA-dataset/master/notebooks/resource/stop_words_ja.txt

--2018-11-09 13:18:57--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.0.80
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.0.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K  --.-KB/s    in 0.09s   

2018-11-09 13:18:57 (7.88 MB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

e00008_ann.json  e01118_ann.json  e02289_ann.json  e04291_ann.json
e00017_ann.json  e01151_ann.json  e02353_ann.json  e04298_ann.json
e00024_ann.json  e01156_ann.json  e02367_ann.json  e04304_ann.json
e00026_ann.json  e01173_ann.json  e02380_ann.json  e04319_ann.json
e00030_ann.json  e01183_ann.json  e02382_ann.json  e04329_ann.json
e00033_ann.json  e01197_ann.json  e02390_ann.json  e04331_ann.json


In [2]:
DATA = Path('chABSA-dataset')

def check_data_existence(folder):
    file_count = len(list(folder.glob("e*_ann.json")))
    if  file_count == 0:
        raise Exception("Processed Data does not exist.")
    else:
        print("{} files ready.".format(file_count))

check_data_existence(DATA)

stop_words = []
with (DATA/"stop_words_ja.txt").open(encoding="utf-8") as f:
    stop_words = f.readlines()
    stop_words = [w.strip() for w in stop_words]

print("{} stop words ready.".format(len(stop_words)))

230 files ready.
310 stop words ready.


In [3]:
labels = []
# make labels (exclude NULL and OOD)
for e in ["market", "company", "business", "product"]:
    for a in ["general", "sales", "profit", "amount", "price", "cost"]:
        labels.append(e + "#" + a)
        if e in ["market"]:
            break;
print(labels)

['market#general', 'company#general', 'company#sales', 'company#profit', 'company#amount', 'company#price', 'company#cost', 'business#general', 'business#sales', 'business#profit', 'business#amount', 'business#price', 'business#cost', 'product#general', 'product#sales', 'product#profit', 'product#amount', 'product#price', 'product#cost']


In [15]:
import json
import numpy as np
import pandas as pd
from collections import Counter

sentences = []
dataset = []
tokenizer = get_mecab_tokenizer(stop_words=stop_words, normalize=False)

for f in DATA.glob("e*_ann.json"):
    with f.open(encoding="utf-8") as j:
        d = json.load(j)
        for s in d["sentences"]:
            tokenized = tokenizer.tokenize(s["sentence"].upper())
            for o in s["opinions"]:
                if o["category"] in labels:
                    # sentence index + category
                    dataset.append((len(sentences), o["category"], o["polarity"]))
            sentences.append(tokenized)

## Polarity classification

In [16]:
from sklearn.model_selection import train_test_split
Y = 2
dataset = np.array(dataset)
Xtrn, Xval, ytrn, yval = train_test_split(dataset[:, 0], dataset[:, Y], test_size=0.1, random_state=0)

def write_dataset(filename, X, y):
    with open(filename, 'w') as f:
        for _x, _y in zip(X, y):
            w = ['__label__'+_y]
            w += list(sentences[int(_x)])
            f.write(' '.join(w)+'\n')
write_dataset(DATA/'train.txt', Xtrn, ytrn)
write_dataset(DATA/'valid.txt', Xval, yval)
len(Xtrn), len(Xval), list(set(dataset[:, Y]))

(2381, 265, ['negative', 'neutral', 'positive'])

In [17]:
! fasttext supervised -input {DATA}/train.txt -output {DATA}/ft_supervised -dim 50 -epoch 10000

Read 0M words
Number of words:  3705
Number of labels: 3
Progress: 100.0% words/sec/thread: 2769147 lr:  0.000000 loss:  0.209850 ETA:   0h 0m 0.237838 ETA:   0h 0m  0h 0m


In [18]:
! fasttext test {DATA}/ft_supervised.bin {DATA}/valid.txt 1

N	265
P@1	0.751
R@1	0.751
Number of examples: 265


## Category classification

In [8]:
from sklearn.model_selection import train_test_split
Y = 1
dataset = np.array(dataset)
Xtrn, Xval, ytrn, yval = train_test_split(dataset[:, 0], dataset[:, Y], test_size=0.1, random_state=0)

def write_dataset(filename, X, y):
    with open(filename, 'w') as f:
        for _x, _y in zip(X, y):
            w = ['__label__'+_y]
            w += list(sentences[int(_x)])
            f.write(' '.join(w)+'\n')
write_dataset(DATA/'train.txt', Xtrn, ytrn)
write_dataset(DATA/'valid.txt', Xval, yval)
len(Xtrn), len(Xval), list(set(dataset[:, Y]))

(2381,
 265,
 ['product#profit',
  'business#price',
  'company#general',
  'business#profit',
  'business#amount',
  'product#amount',
  'product#price',
  'product#general',
  'company#amount',
  'business#sales',
  'company#cost',
  'company#profit',
  'market#general',
  'business#cost',
  'product#cost',
  'business#general',
  'company#sales',
  'product#sales'])

In [9]:
! fasttext supervised -input {DATA}/train.txt -output {DATA}/ft_supervised -dim 50 -epoch 10000

Read 0M words
Number of words:  3705
Number of labels: 18
Progress: 100.0% words/sec/thread: 2313320 lr:  0.000000 loss:  0.431946 ETA:   0h 0m 0.457045 ETA:   0h 0m 0.439391 ETA:   0h 0mh 0m


In [10]:
! fasttext test {DATA}/ft_supervised.bin {DATA}/valid.txt 1

N	265
P@1	0.464
R@1	0.464
Number of examples: 265
